In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arc import *
from time import time
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from typing import List, Dict

from basics import *
from utility import *
from floquet_hamiltonian import *

pi=np.pi
tau=2*pi

# Plot old measured polarizabilities

In [ ]:
exp_pol = pd.read_csv("Pin1_4780MHz_Polarizabilities.csv")

In [ ]:
exp_pol

In [ ]:
x_dat = exp_pol["RF power (dBm)"]
y_str = "Effective Polarizability {} (MHz/V^2)"
er_str = "Error {} (MHz/V^2)"
fig, ax = plt.subplots(1,1)
for band in range(4):
    y_dat = exp_pol[y_str.format(band)]
    y_err = exp_pol[er_str.format(band)]
    ax.errorbar(x_dat,y_dat,yerr=y_err,fmt=".",label = f"Band {band}")
ax.legend()
ax.set_xlabel("RF power (dBm)")
ax.set_ylabel("Measured Effective Polarizability ($MHz/V^2$)")
fig.show()

In [ ]:
x_dat_dB = exp_pol["RF power (dBm)"]
x_dat_scale = np.sqrt(10**(x_dat_dB/10))
y_str = "Effective Polarizability {} (MHz/V^2)"
er_str = "Error {} (MHz/V^2)"
fig, ax = plt.subplots(1,1)
for band in range(4):
    y_dat = exp_pol[y_str.format(band)]
    y_err = exp_pol[er_str.format(band)]
    ax.errorbar(x_dat_scale+band*2e-4,y_dat,yerr=y_err,fmt=".",label = f"Band {band}")
ax.legend()
ax.set_xlabel("RF power (Proportional to $(V/m)^2$)")
ax.set_ylabel("$-2\\alpha'$ on Pin 1 ($MHz/V^2$)")
fig.show()

# Load AC stark shift datasets

In [ ]:
# Load datasets

# Taken with 684nm +80MHz from 4-6' transition
dAC_1 = pd.read_csv("ExperimentData/AC_stark_m30-m23dBm-m50MHz.csv")
dAC_2 = pd.read_csv("ExperimentData/AC_stark_m23dBm-m10dBm-m50MHz.csv")

# Taken with 684nm +60MHz from 4-6' transition
'''
# these datasets excluded to make use of files with GOOD data included
dAC_4 = pd.read_csv("ExperimentData/AC_stark_m10dBm-m6dBm-m50MHz-2022-01-24.csv")
dAC_5 = pd.read_csv("ExperimentData/AC_stark_m30dBm-m25dBm-m50MHz-2022-01-25.csv")
dAC_6 = pd.read_csv("ExperimentData/AC_stark_m25dBm-m18dBm-m50MHz-2022-01-26.csv")
'''
# GOOD logger inclusive versions of above datasets
dAC_4 = pd.read_csv("ExperimentData/AC_stark_m16dBm-m8dBm-m50MHz-2022-01-24-GOOD.csv")
dAC_5 = pd.read_csv("ExperimentData/AC_stark_m30dBm-m25dBm-m50MHz-2022-01-25-GOOD.csv")
dAC_6 = pd.read_csv("ExperimentData/AC_stark_m25dBm-m18dBm-m50MHz-2022-01-26-GOOD.csv")
dAC_7 = pd.read_csv("ExperimentData/AC_stark_m12dBm-m6dBm-m50MHz-2022-01-27-GOOD.csv")

dAC_m20 = dAC_5.append(dAC_6)
dAC_m20 = dAC_m20.append(dAC_4[:-2])
dAC_m20 = dAC_m20.append(dAC_7)

dAC_m20[:-5]

drive_cal = pd.read_csv("ExperimentData/4780MHZ_pickoffPower.csv")

In [ ]:
# Calibrate drive power
x_set = np.array(drive_cal["Set Power (dBm)"])
y_pk = np.array(drive_cal["Pick-off power (dBm)"])

cal_fun = interp1d(x_set, y_pk, kind="linear")

In [ ]:
# Failed GOOD logger version of calibration
fn = lambda x, a, s: a*10**(float(s)*x)
fig,ax = plt.subplots(1,1)
ax.plot(dAC_m20["RydDressPower (dBm)"], np.abs(dAC_m20["Measured Dressing Power (arb)"]),".")
xln = np.linspace(min(dAC_m20["RydDressPower (dBm)"]), max(dAC_m20["RydDressPower (dBm)"]), 1000)
ax.plot(dAC_m20["RydDressPower (dBm)"],fn(dAC_m20["RydDressPower (dBm)"],100,0.07))
ax.set_xlabel("Set Power (dBm)")
ax.set_ylabel("Measured Power (arb)")
ax.set_yscale("log")
fig.show()

In [ ]:
# Build dataframe that concatenates all data taken at +80MHz intermediate detuning

x_ignores = 2  # ignoring the first n values from the dataset in dAC_2
x_data = np.array(dAC_1["RydDressPower (dBm)"])
x_data = np.append(x_data,dAC_2["RydDressPower (dBm)"][x_ignores:])
print(x_data)

n_bands = 4
dACs = np.zeros((n_bands,len(x_data)),dtype=float)
dACs_er = np.zeros(dACs.shape,dtype=float)

# add data from dataset 1
for band in range(n_bands):
    dACs[band,:len(dAC_1)] = dAC_1[f"Band {band//2} (MHz)"]
    dACs_er[band,:len(dAC_1)] = dAC_1[f"Band {band//2} unc (MHz)"]
    
print(dACs)
dACs = dACs[::-1,:]
# add data from dataset 2
for band in range(n_bands):
    dACs[band,len(dAC_1):] = dAC_2[f"Band {band} (MHz)"][x_ignores:]
    dACs_er[band,len(dAC_1):] = dAC_2[f"Band {band} unc (MHz)"][x_ignores:]

x_data_80 = x_data
dACs_80 = dACs
dACs_80_er = dACs_er

print(dACs_80)

In [ ]:
# Build dataset that concatenates all data taken at +60MHz intermediate detuning

x_data_60 = np.array(dAC_m20["RydDressPower (dBm)"])
x_data_lin = np.abs(np.array(dAC_m20["Measured Dressing Power (arb)"]))
x_lin_std = np.array(dAC_m20["Measured Dressing Power std (arb)"])
dACs_60 = np.zeros((n_bands,len(x_data_60)),dtype=float)
dACs_60_er = np.zeros(dACs_60.shape, dtype=float)
for band in range(n_bands):
    dACs_60[band] = dAC_m20[f"Band {band} (MHz)"]
    dACs_60_er[band] = dAC_m20[f"Band {band} unc (MHz)"]
    
x_cal = cal_fun(x_data_60)

In [ ]:
# Select between +60MHz and +80MHz intermediate detuning datasets for all future plotting

dataset = 60  # 80
if dataset == 60:
    last_n = 0
    # exclude last n datapoints for fits
    if last_n > 0:
        x_data = x_cal[:-last_n] # x_data_60[:]
        dACs = dACs_60[:,:-last_n]
        dACs_er = dACs_60_er[:,:-last_n]
        # fraction of dataset excluded from fits to verify extrapolation capability of model
        x_ver = x_cal[-last_n:]#x_data_60[-5:]
        dACs_ver = dACs_60[:,-last_n:]
        dACs_ver_er = dACs_60_er[:,-last_n:]
    else:
        vn = 8
        x_data = x_cal # x_data_60[:]
        dACs = dACs_60
        dACs_er = dACs_60_er
        # fraction of dataset excluded from fits to verify extrapolation capability of model
        x_ver = x_cal[-vn:]
        dACs_ver = dACs_60[:,-vn:]
        dACs_ver_er = dACs_60_er[:,-vn:]
elif dataset == 80:
    x_data = x_data_80
    dACs = dACs_80
    dACs_er = dACs_80_er

In [ ]:
dACs_ver.shape

In [ ]:
# Plot AC stark shift data and extracted CoG AC stark shift
fig, ax = plt.subplots(1,1)
for band in range(n_bands):
    ax.errorbar(x_data,dACs[band],yerr=dACs_er[band],label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
ax.errorbar(x_data,dAC_com,yerr=dAC_com_er)
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
fig.show()

In [ ]:
# PLot +60MHz AC stark shifts against measured linearized power
# Failed Attempt! Remove during refactor
fig, ax = plt.subplots(1,1)
for band in range(n_bands):
    ax.errorbar(x_data_lin,dACs[band],yerr=dACs_er[band],xerr=x_lin_std,label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
ax.errorbar(x_data_lin,dAC_com,yerr=dAC_com_er)
ax.set_xlabel("Measured Dressing Power (arb)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
#ax.set_xscale("log")
fig.show()

# Fit CoG AC stark shift to expected behavior

In [ ]:
# dB data
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)

fig, ax = plt.subplots(1,1)
#for band in range(n_bands):
#    ax.errorbar(x_data_lin,dACs[band],yerr=dACs_er[band],xerr=x_lin_std,label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
guess = [-600, 0.2]

xls = np.linspace(min(x_data),max(x_data),1000)
ax.errorbar(x_data,dAC_com,yerr=dAC_com_er, label="experiment data")
ax.plot(xls, func(xls,*guess), label="fit")
ax.set_xlabel("Measured Dressing Power (arb)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
ax.legend()
#ax.set_xscale("log")
fig.show()

In [ ]:
# Use raster scan and rms of residuals to get a rough least-squares fit of data to model

def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

# dB data
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
# linear data
#func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2) - c)

xdat = x_data
ydat = dAC_com
yer = dAC_com_er
alphas = -np.linspace(625,680,300)
cs = np.linspace(0.25,0.34,300)
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [ ]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
#    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,
    max(alphas)+alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)

im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent , vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+1)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

In [ ]:
# extract optimal fitting paramters
good_inds = np.argwhere(residuals_rms == np.min(residuals_rms))[0]
fits = alphas[good_inds[0]],cs[good_inds[1]]
print(good_inds)
print(fits)

In [ ]:
# plot data against CoG fit
#func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
inv_cal = interp1d(y_pk, x_set, kind="linear")

guess = fits

fig,axar = plt.subplots(2,1,figsize=(5,5.5),gridspec_kw={'height_ratios': [3, 1]})#,sharex=True)

ax=axar[0]
ax.errorbar(inv_cal(xdat),ydat,yerr=yer,fmt=".")
xlin=np.linspace(min(xdat),max(xdat),1000)
ax.plot(inv_cal(xlin),func(xlin,*guess))
ax.set_title(f"Experiment Data Vs Fit\n$\Delta_{{ac}} = {guess[0]:.0f}*(\sqrt{{{guess[1]:.3f}^2+10^{{P/10}} }} - {guess[1]:.3f})$")
#ax.set_title(f"Experiment Data Vs Fit\n$\Delta_{{ac}} = {guess[0]:.0f}*(\sqrt{{{guess[1]:.3f}^2+P^2}} - {guess[1]:.3f})$")
#ax.set_xlabel("Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power Measured (arb)")
ax.set_ylabel("$\Delta_{ac} (MHz)$")

ax=axar[1]
ax.errorbar(inv_cal(xdat),ydat-func(xdat,*guess),yerr=yer,fmt=".")
ax.axhline(0,ls=":",c="tab:orange")
#ax.set_title("Fit Residuals")
ax.set_ylabel("Residuals (MHz)")
ax.set_xlabel("Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power Measured (arb)")
#ax.plot(xlin,func(xlin,*guess))
axar[0].get_shared_x_axes().join(axar[0], axar[1])
axar[0].set_xticklabels([])
axar[1].set_xlim(axar[0].get_xlim())

for ax in axar:
    for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
        txt.set_fontsize("12")
fig.tight_layout()
fig.savefig("Plots/UnitlessCoMFit.png")
fig.show()

In [ ]:
# estimate sensitivity to a 0.1dBm miscalibration of Dressing Power
a = fits[0]
c = fits[1]
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
dfunc = lambda x, alpha, c: -alpha*2**(x/10-2)*5**(x/10-1)*np.log(10)/np.sqrt(c**2+10**(x/10))

sens = 0.1*dfunc(-10.289,1589,0.095) # MHz

# Scan Eac to match up numerical estimates with experimental data

In [ ]:
levels, basis, basis_n = None, None, None
ellipticity, scale = None, None

In [ ]:
fields = {
    "Eac": np.append(np.linspace(0,15,200),np.linspace(15+75/300,90,350)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": 0.012,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 2
n_max = 2

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)
    
t_inds = [ind for ind, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]

# measured properties

# for 685nm at +60MHz wrt 4-6' transition. Generator at 181.25MHz
if dataset == 60:
    # measured resonance between nP3/2 -> nD3/2 levels
    p3o2_d3o2_res = 4839*1e6*tau
# for 685nm at +80MHz wrt 4-6' transition. Generator at 191.25MHz
elif dataset == 80:
    p3o2_d3o2_res = 4840*1e6*tau
# effective laser induced AC stark shift
laser_dAC = detuning(RydStateFS(51,2,3/2), t_level, p3o2_d3o2_res)

## Single Scan

In [ ]:
#ellips = arange(0.015,0.024,0.001)
#scales = linspace(150,180,100)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j
#for ellip in ellips: 
#    fields["ellipticity"] = ellip
try:
    del energies
    del eigenstates
except NameError:
    pass
energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

#    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#    dAC_num = energies
    
#    for scale in scales:
        

## Iterative scans
Repeat raster-scan to least-squares fit technique from CoG fit above

In [ ]:
ellips = np.arange(0.01125,0.0125,0.00025)
scales = np.arange(63,65,0.01)
#scales=[167]
residuals = np.zeros((len(ellips),len(scales)), dtype=float)
residuals_rms = np.zeros(residuals.shape, dtype=float)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#Reorder experiment bands
tmp = basis_t[1]
basis_t[1] = basis_t[2]
basis_t[2] = tmp
basis_t = basis_t[1:] + basis_t[:1]
basis_t = basis_t[::-1]

for i, ellip in enumerate(ellips):
    try:
        del energies
        del eigenstates
    except NameError:
        pass
    fields["ellipticity"] = ellip
    energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

    dAC_num = energies
    
    for j, scale in enumerate(scales):
        x_ac = 20*np.log10(varied[1]/scale)
        #fig, axar = plt.subplots(1,2,figsize=(12,6))
        #ax=axar[0]
        #ax1=axar[1]
        for band in range(n_bands):
            y_num = 1e-6*(energies[basis_t[::-1][band]]-energies[basis_t[::-1][band],0])/tau
            y_ex = dACs[band]
            dy_er = dACs_er[band]
            #ax.plot(x_ac,y_num,label=f"numerical band {band}")
            #ax.errorbar(x_data,y_ex,yerr=dy_er,fmt=".",label=f"experimental band {band}")
            y_int = np.interp(x_data,x_ac,y_num)
            res = y_ex-y_int
            #ax1.errorbar(x_data,res,yerr=dy_er,fmt=".",label=f"Band {band}")
            residuals[i,j] += res.mean()
            residuals_rms[i,j] = np.sqrt(residuals_rms[i,j]**2+(res**2).sum())
        '''ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.3f}; V_{{ref}} = {scale:.0f}V/m$")
        ax.set_xlabel("Dressing Power (dBm)")
        ax.set_ylabel("$\Delta_{ac}-CoM (MHz)$")
        ax.set_xlim(-31,-9)
        ax.legend(loc=3)
        ax1.set_title("Fit residuals")
        ax1.set_xlabel("Dressing Power (dBm)")
        ax1.set_ylabel("Fit Residuals (MHz)")
        fig.show'''

In [ ]:
fig,ax = plt.subplots(1,1)
eps_step = (max(ellips)-min(ellips))/len(ellips)
scale_step = (max(scales)-min(scales))/len(scales)
extent = [
    min(scales)-scale_step/2,
    max(scales)+scale_step/2,
    max(ellips)+eps_step/2,
    min(ellips)-eps_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("$V_{ref}$")
ax.set_ylabel("$\epsilon$")
ax.set_xticks(scales.astype(float),minor=2)
ax.set_yticks(ellips.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+1)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("$V_{ref}$")
ax.set_ylabel("$\epsilon$")
ax.set_xticks(scales.astype(float),minor=2)
ax.set_yticks(ellips.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

In [ ]:
# Extract fit parameters
good_inds = np.argwhere(residuals_rms == np.min(residuals_rms))[0]
ellipticity = float(ellips[good_inds[0]])
scale = float(scales[good_inds[1]])
print(np.argwhere(residuals_rms == np.min(residuals_rms)))
print(ellipticity, scale)
# delete large residuals arrays to make space for final scan
del residuals
del residuals_rms

In [ ]:
# manually enter optimal fit values if known, to avoid running raster scan each time
if dataset == 60:
    # ellipticity = 0.0135
    if ellipticity is None:
        ellipticity = 0.0120
    if scale is None:
        scale = 64.1  # V/m @ 0dBm
elif dataset == 80:
    if ellipticity is None:
        ellipticity = 0.0245
    if scale is None:
        scale = 167.822  # V/m @ 0dBm
fields = {
    "Eac": np.append(np.linspace(0,15,200),np.linspace(15+75/300,90,350)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": ellipticity,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 2
n_max = 2

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)

## Single Scan

In [ ]:
# Re-run 1D scan using optimized fit paramters
try:
    del energies
    del eigenstates
except NameError:
    pass

#ellips = arange(0.015,0.024,0.001)
#scales = linspace(150,180,100)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j

#for ellip in ellips:
#    fields["ellipticity"] = ellip
energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

#    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#    dAC_num = energies
    
#    for scale in scales:
        

In [ ]:
# create plot of experiment data and fitted numerical data

inv_cal = interp1d(y_pk, x_set, kind="linear")
colors = ['tab:blue','tab:orange', 'tab:green', 'tab:red']

fig,axar = plt.subplots(2,1,figsize=(5,5.5),gridspec_kw={'height_ratios': [3, 1]})#,sharex=True)
ax = axar[0]
for band in range(n_bands):
    lab = f"experiment" if band == 0 else ""
    #lab = f"experimental {band}"
    ax.errorbar(
        inv_cal(x_data)[:-vn],
        (dACs[band]-func(x_data,*fits))[:-vn],
        yerr = dACs_er[band][:-vn],
        fmt = ".", 
        label=lab,
        c=colors[band]
    )
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

#y = 0.6
#x_data_3_p = x_data_3+np.log10(1+y*10**-(x_data_3/20))
#for band in range(n_bands):
#    lab = "verification data" if band == 0 else ""
#    ax.errorbar(x_data_3_p,dACs3[band]-func(x_data_3_p,*fits)+10*0-22, yerr = dACs3_er[band], fmt="o", label=lab)

if dataset == 60:
    x_data_2 = x_ver
    dACs_2 = dACs_ver
    dACs_2_er = dACs_ver_er
elif dataset == 80:
    x_data_2 = x_data_60
    dACs_2 = dACs_60
    dACs_2_er = dACs_60_er
ms=8
plot_vers = True
if plot_vers:
    y = 0.0
    #x_data_2_p = x_data_2+np.log10(1+y*10**-(x_data_2/20))
    for band in range(n_bands):
        # lab = "verification data" if band == 0 else ""
        ax.errorbar(
            inv_cal(x_data_2),
            dACs_2[band]-func(x_data_2,*fits),
            yerr = dACs_2_er[band], 
            fmt=".",
            label=lab,
            fillstyle="none",
            c=colors[band],
            markersize=ms
        )

    
thrs = 0.10


basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]

#Reorder experiment bands
tmp = basis_t[1]
basis_t[1] = basis_t[2]
basis_t[2] = tmp
basis_t = basis_t[1:] + basis_t[:1]

interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
x_ac = 20*np.log10(varied[1]/scale)
interp_inds = np.argwhere((x_ac > min(y_pk))*(x_ac < max(y_pk))).T[0]
x_acp = x_ac[interp_inds]
energiesp = energies[:,interp_inds]
for i,k in enumerate(basis_t):
#for i,k in enumerate(interesting_inds[::-1]): 
    lab = f"fit" if i==0 else ""
    #lab = f"fit {i}"
    print(k)
    print(colors[i])
    ax.plot(inv_cal(x_acp), 1e-6*(energiesp[k]-energies[k,0])/tau-func(x_acp,*fits), ls=":", label=lab, c=colors[i])
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
#ax.axvline(inv_cal(20*np.log10(25.5/scale)),ls=":",label="Expected Polarizability minumum")
zero_xs = [78.6,82.6,85.6,89.6]
for i,zerox in enumerate(zero_xs):
    ax.axvline(inv_cal(20*np.log10(zerox/scale)),ls="--",c=colors[i],linewidth=1)#,label="Polarizability minumum")
    print(f"zero_x : {inv_cal(20*np.log10(zerox/scale))}dBm")

#ax.axvline()

ax.set_xlim(-30.5,-5.25)
ax.set_ylim(-88,66)

ax.legend(loc=3)
#ax.set_xlabel("Calibrated Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ - CoG (MHz)")
ellip = fields["ellipticity"]
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
#ax.set_title(f"AC stark shift fit - CoG\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("11")

ax = axar[1]
for band in range(n_bands):
    y_interp = np.interp(x_data,x_ac, 1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
    '''
    if band == 1:
        band = 2
    elif band == 2:
        band = 1
    '''
    res = dACs[band] - y_interp
    ax.errorbar(inv_cal(x_data)[:-vn],res[:-vn],yerr=dACs_er[band][:-vn],fmt=".",label=f"band {band}", c=colors[band])
    if plot_vers:
        y_interp = np.interp(x_data_2,x_ac,1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
        lab = "verification data" if band == 0 else ""
        ax.errorbar(inv_cal(x_data_2),dACs_2[band]-y_interp, yerr = dACs_2_er[band], fmt=".", label=lab, fillstyle="none", c=colors[band],markersize=ms)
ax.set_ylim(-6,6)
ax.set_xlim(-30.5,-5.5)
#ax.legend(loc=3)
ax.axhline(0,ls=":",c="r")

#ax.set_xlabel("Calibrated Dressing Power (dBm)")

ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Residuals (MHz)")
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

axar[0].get_shared_x_axes().join(axar[0], axar[1])
axar[0].set_xticklabels([])
axar[1].set_xlim(axar[0].get_xlim())
#axar[0].set_xlim
#ax.set_title(f"AC stark shift fit residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V_{{cal}}/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("10")

fig.tight_layout()
fig.show()
fig.savefig("Calibrated AC stark map.png")

# Legacy
## ALL CODE BELOW THIS POINT IS NOT IN ACTIVE USE, KEPT AROUND FOR REMINDERS OF SYNTAX AND DEBUGGING

In [ ]:
thrs = 0.5
fig, ax = plt.subplots(1,1, figsize=(5,5))

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
scale = 79.5/0.474
x_ac = 20*np.log10(varied[1]/scale)
for k in interesting_inds:
    ax.plot(x_ac, 1e-6*np.real(energies[k,:]-energies[k,0]*0)/tau,label = basis[k%len(basis)].ket())

scale = 79.5/0.474
x_ac = 20*np.log10(varied[1]/scale)
for band in range(n_bands):
    ax.errorbar(x_data, dACs[band], yerr = dACs_er[band], fmt = ".", label=f"band {band}")

ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$; $\epsilon = {fields['ellipticity']:.3f}$; $P_{{scale}}={scale:.0f}V/m$")
ax.set_ylabel("$\Delta_{AC}(MHz)$")
ax.set_xlabel("Dressing Power (dBm)")
#ax.set_ylim(-900,-80)
ax.set_xlim(-20,max(x_ac))
#ax.set_xscale("log")
#ax.set_yscale("symlog")
#ax.set_title(f"Bands wrt Ellipticity\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\nEac = {fields['Eac']:.3f} V/m\n")
#ax.set_ylabel(f"Band Energy - $E_{{{t_level.ket()}}}$ (MHz)")
#ax.set_xlabel("Dressing Field Ellipticity")
ax.legend()
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()
#fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")#
#fig.savefig(f"Band Frequency ellipticity.png")

In [ ]:

fits = [alphas[42],cs[67]]
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
xdat = x_data_dBm[1:]

fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Mean AC stark shift - Fit value (MHz)")
ax.set_title(f"Fit Residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()

fig.show()

print(fits)

In [ ]:
20*np.log10(55/scale)

In [ ]:
dACs.mean[1]

def abs_split(ar):
    return abs(max(ar)-min(ar))
dAC_CoM = (np.array([np.nanmean([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])-bare_res)
dAC_CoM_er = np.array([np.sum([row[er_str.format(band)]**2 for band in bands]) for ind,row in res_df.iterrows()])
dAC_split = np.array([abs_split([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(x_data_dBm[1:],dAC_CoM[1:],yerr=dAC_CoM_er[1:],fmt=".")
ax.plot(x_data_dBm[1:],dAC_CoM[1:]+dAC_split[1:]/2)
ax.plot(x_data_dBm[1:],dAC_CoM[1:]-dAC_split[1:]/2)
ax.set_title("COM and Splitting verify")
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
fig.show()

In [ ]:
res_df = pd.read_csv("Measured_RF_Resonances_4780.csv")
#res_df = pd.read_csv("Measured_RF_Resonances_4780_2.csv")
res_df

In [ ]:
bands = range(1,6)
x_data_dBm = res_df["RF power (dBm)"]
x_data_Vs = np.sqrt(10**(x_data_dBm/10))
y_str = "mu {} (MHz)"
er_str = "sig {} (MHz)"
bare_res = res_df["mu 1 (MHz)"][0]
fig, ax = plt.subplots(1,1, figsize=(5,5))
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = 2*(measured_res-bare_res)
    dAC_er = 2*errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
ax.set_yscale("symlog")
ax.set_title("Measured AC stark shifts\n4780MHz Dressing field")
ax.set_xlabel("Drive Power (dBm)")
ax.set_ylabel("$-\Delta_{AC} (MHz)$")
fig.tight_layout()
fig.show()
# fig.savefig("Unscaled Wiki AC stark shifts.png")

In [ ]:
def abs_split(ar):
    return abs(max(ar)-min(ar))
dAC_CoM = (np.array([np.nanmean([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])-bare_res)
dAC_CoM_er = np.array([np.sum([row[er_str.format(band)]**2 for band in bands]) for ind,row in res_df.iterrows()])
dAC_split = np.array([abs_split([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(x_data_dBm[1:],dAC_CoM[1:],yerr=dAC_CoM_er[1:],fmt=".")
ax.plot(x_data_dBm[1:],dAC_CoM[1:]+dAC_split[1:]/2)
ax.plot(x_data_dBm[1:],dAC_CoM[1:]-dAC_split[1:]/2)
ax.set_title("COM and Splitting verify")
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
fig.show()

In [ ]:
xdat = x_data_Vs[1:]
ydat = dAC_CoM[1:]
yer = dAC_CoM_er[1:]

func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2)-c)
guess= [-1.76e3/2,1.4e-1]

popt,pcov = curve_fit(func,xdat,ydat,sigma=yer,p0=guess)
perr = np.sqrt(np.diag(pcov))

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func(xlin,*popt))
ax.plot(xlin,func(xlin,*guess))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*guess),yerr=yer,fmt=".")
fig.show()

In [ ]:
def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

alphas = np.linspace(-1.825,-1.55,100)*1e3/2
cs = np.linspace(0.8,1.3,100)*1e-1
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [ ]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+10)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

In [ ]:
np.argwhere(residuals_rms==np.min(residuals_rms))[0]

In [ ]:
fits = [alphas[42],cs[67]]
func2 = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
xdat = x_data_dBm[1:]

fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Mean AC stark shift - Fit value (MHz)")
ax.set_title(f"Fit Residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()

fig.show()

print(fits)

In [ ]:
fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    
    ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

x_ac = 20*np.log10(varied[1]/scale)
for k in basis_t:
    ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits))
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlim(-20,max(x_ac))
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

In [ ]:
xdat = x_data_Vs[1:]
ydat = dAC_split[1:]
yer = dAC_CoM_er[1:]

func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2)-c)
guess= [4.76e2, 2e-1]

popt,pcov = curve_fit(func,xdat,ydat,sigma=yer,p0=guess)
perr = np.sqrt(np.diag(pcov))

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
#ax.plot(xlin,func(xlin,*popt))
ax.plot(xlin,func(xlin,*guess))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*guess),yerr=yer,fmt=".")
fig.show()

In [ ]:
def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

alphas = np.linspace(2.2,3.1,100)*1e2/2
cs = np.linspace(0.05,0.12,100)
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [ ]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+10)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.tight_layout()
fig.show()

In [ ]:
np.argwhere(residuals_rms==np.min(residuals_rms))[0]

In [ ]:
fits = [alphas[49],cs[46]]

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func(xlin,*fits))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*fits),yerr=yer,fmt=".")
fig.show()

In [ ]:
print(fits)

In [ ]:
func2 = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)

fits = [-854.17,0.1138]
xdat = x_data_dBm[1:]
ydat = dAC_CoM[1:]
yer = dAC_CoM_er[1:]

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
fig.show()

In [ ]:
func2(-30,*fits)+40.5

In [ ]:
bands = range(1,6)
x_data_dBm = res_df["RF power (dBm)"]
x_data_Vs = np.sqrt(10**(x_data_dBm/10))
y_str = "mu {} (MHz)"
er_str = "sig {} (MHz)"
bare_res = res_df["mu 1 (MHz)"][0]
fig, ax = plt.subplots(1,1, figsize=(8,8))
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = 2*(measured_res-bare_res)
    dAC_er = 2*errb
    ax.errorbar(x_data_Vs, dAC, yerr = dAC_er, fmt = ".")
ax.set_title("AC stark shifts on $|52P_{3/2}$ induced by 4780MHz Dressing field")
ax.set_xlabel("Dressing Field Strength (arb)")
ax.set_ylabel("AC stark shift on $|52P_{3/2}>$ band (MHz)")
fig.tight_layout()
fig.show()
fig.savefig("Unscaled Wiki AC stark shifts.png")

In [ ]:
energies[basis_t].mean(0)

In [ ]:
levels, basis, basis_n = None, None, None

In [ ]:
fields = {
    "Eac": np.append(np.linspace(0,22,200), np.linspace(22+22/200,100,200)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": 0.023,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 1
n_max = 3

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)

In [ ]:
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, **fields)

In [ ]:
thrs = 0.5
fig, ax = plt.subplots(1,1, figsize=(5,5))

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
scale = 79.5/0.474
for k in interesting_inds:
    ax.plot(x_ac, 1e-6*np.real(energies[k,:]-energies[k,0]*0)/tau,label = basis[k%len(basis)].ket())

scale = 79.5/0.474
for band in range(n_bands):
    ax.errorbar(x_data, dACs, yerr = dACs_er, fmt = ".", label=f"band {band}")

ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$; $\epsilon = {fields['ellipticity']:.3f}$; $P_{{scale}}={scale:.0f}V/m$")
ax.set_ylabel("$\Delta_{AC}(MHz)$")
ax.set_xlabel("Dressing Power (dBm)")
#ax.set_ylim(-900,-80)
ax.set_xlim(-20,max(x_ac))
#ax.set_xscale("log")
#ax.set_yscale("symlog")
#ax.set_title(f"Bands wrt Ellipticity\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\nEac = {fields['Eac']:.3f} V/m\n")
#ax.set_ylabel(f"Band Energy - $E_{{{t_level.ket()}}}$ (MHz)")
#ax.set_xlabel("Dressing Field Ellipticity")
ax.legend()
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()
#fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")#
#fig.savefig(f"Band Frequency ellipticity.png")

In [ ]:
fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    if band == 1:
        ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".", label=f"measured resonances")
    else:
        ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

scale = 79.5/0.473
x_ac = 20*np.log10(varied[1]/scale)
for k in basis_t:
    if k == min(basis_t):
        ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits), label = "computed resonances")
    else:
        ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits))
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlim(-20,max(x_ac))
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC,i}$ - Mean (MHz)")
field_omega=fields["field_omega"]
ellipticity = fields["ellipticity"]
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot {field_omega*1e-6/tau:.0f}MHz$; $\epsilon = {ellipticity}$; $P_{{scale}} = {scale:.0f}V/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

In [ ]:
k
len()

In [ ]:
eps = np.arange(0,0.045,0.005)
for ellip in eps:
    ellip = float(ellip)
    fields = {
        "Eac": np.append(np.linspace(0,22,200), np.linspace(22+22/200,100,200)),
        "ellipticity": ellip,
        "field_omega": tau*4780e6,
        "Edc": 0.0,
        "theta": 0
    }

    for key, value in fields.items():
        try:
            tmp = value[1]
        except TypeError:
            continue
        else:
            varied = (key, value)

    H0 = build_H0(basis, t_level, offset_zeeman=1e6)
    energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, **fields)

    thrs = 0.5
    fig, ax = plt.subplots(1,1, figsize=(8,8))

    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
    interesting_inds = []
    for t_ind in basis_t:
        for k, ps in enumerate(eigenstates[:, t_ind, :]):
            if max(np.absolute(ps)**2) > thrs:
                interesting_inds.append(k)

    interesting_inds = list(set(interesting_inds))
    for k in interesting_inds:
        ax.plot(varied[1], 1e-6*np.real(energies[k,:]-energies[k,0])/tau, label = basis_n[k].ket())

    scale = 79/0.474
    for band in bands:
        measured_res = res_df[y_str.format(band)]
        errb = res_df[er_str.format(band)]
        dAC = 2*(measured_res-bare_res)
        dAC_er = 2*errb
        x_data_scaled = x_data_Vs*scale
        ax.errorbar(x_data_scaled, dAC, yerr = dAC_er, fmt = ".", label=f"band {band}")

    ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\n$\epsilon = {fields['ellipticity']:.3f}$")
    ax.set_ylabel("AC stark shift (MHz)")
    ax.set_xlabel("AC field strength (V/m)")
    ax.legend()
    fig.tight_layout()
    fig.show()
    fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")

In [ ]:
key

In [ ]:
type(value)

In [ ]:
basis_t

In [ ]:
basis[basis_t[3]].ket()